In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('diabetes.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'diabetes.csv'

In [ ]:
df.head()

In [ ]:
df.corr()['Outcome']

In [ ]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()

In [ ]:
x=scaler.fit_transform(x)

In [ ]:
x.shape

In [ ]:
from sklearn.model_selection import train_test_split
x_train ,x_test, y_train , y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [ ]:
model= Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,batch_size=32,epochs=10,validation_data=(x_test,y_test))

### 1.how to select appropriate optimizer
### 2.number of nodes in layer
### 3.how to select number of layers
### 4.All in all one model

# 1.how to select appropriate optimizer

In [ ]:
# pip install -U keras-tuner

In [ ]:
import keras_tuner as kt

In [ ]:
def build_model(hp):  ## hp = hyper parameter
    model=Sequential()
    model.add(Dense(32, activation='relu',input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
#     model.compile(optimize=hp.Choice('optimizer',['adam','sgd','rmsprop','adadelta'],,loss='binary_crossentropy',metrics=['accuracy']))   
#  Can also wrrite as...
    optimizer=hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [ ]:
tuner = kt.RandomSearch(build_model, #function name
                       objective='val_accuracy',  #objective what you wnat to do
                       max_trials=5,
                       directory='mydir1(CampusX_39 Keras Tuner for Hyperparameter Tuning)',
                       project_name='p1')

In [ ]:
#There are may features in tuner object
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))
#initial epoch means it start with previouses epochs

In [ ]:
#here we get 79.22% accuracy while rmsprop gives 0.7727 

# 2.number of nodes in Layer

In [ ]:
def build_model(hp):
    
    model=Sequential()
    
    units =hp.Int('units',min_value=8,max_value=128,step=8)
    
    model.add(Dense(units=units,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
              
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model           ## Here we use adam because adam gives best result in this data as shown above

In [ ]:
tuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5,
                     directory='mydir2(CampusX_39 Keras Tuner for Hyperparameter Tuning)',
                     project_name='p2')

In [ ]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model =tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

# 3.how to select number of layers

In [ ]:
def build_model(hp):
    
    model=Sequential()
    model.add(Dense(72,activation='relu',input_dim=8))
    
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)): #two loops one inside other
        model.add(Dense(72,activation='relu'))
        
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model           

In [ ]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=3,
                       directory='mydir3(CampusX_39 Keras Tuner for Hyperparameter Tuning)',
                       project_name='p3')

In [ ]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model =tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

# 4.All in all one model

In [ ]:
def build_model(hp):
    
    model=Sequential() 
    
    counter=0
    
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
      
        if counter == 0:
            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),input_dim=8))
        else:
            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])))
        counter+=1
        
    model.add(Dense(1,activation='sigmoid'))
            
    model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=3,
                       directory='mydir4(CampusX_39 Keras Tuner for Hyperparameter Tuning)',
                       project_name='p4')

In [ ]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(x_train,y_train,epochs=200,initial_epoch=5,validation_data=(x_test,y_test))

### We can also add dropout in 4th

In [ ]:
def build_model(hp):
    
    model=Sequential() 
    
    counter=0
    
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
      
        if counter == 0:
            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),input_dim=8))
            model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:
            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])))
            model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter+=1
        
    model.add(Dense(1,activation='sigmoid'))
            
    model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=3,
                       directory='mydir5(CampusX_39 Keras Tuner for Hyperparameter Tuning)',
                       project_name='p5')

In [ ]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(x_train,y_train,epochs=200,initial_epoch=5,validation_data=(x_test,y_test))